In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import pydicom
import matplotlib.animation as animation
import re
import matplotlib.animation as animation
from IPython.display import HTML
import os
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from tqdm import tqdm
from PIL import Image
import cv2
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input, Dense, Flatten, ZeroPadding2D, Conv2D, BatchNormalization, Activation, MaxPooling2D, AveragePooling2D, add, concatenate, Dropout
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model 
import tensorflow as tf
import random
from tensorflow.keras.utils import Sequence
from tensorflow.keras.callbacks import  ReduceLROnPlateau
import warnings
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly as py
init_notebook_mode(connected=True)
warnings.filterwarnings("ignore")

C:\Users\bbaah\anaconda3\envs\tf_env\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

C:\Users\bbaah\anaconda3\envs\tf_env\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

C:\Users\bbaah\anaconda3\envs\tf_env\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

C:\Users\bbaah\anaconda3\envs\tf_env\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it wi

In [2]:
IMG_DIR_TRAIN = "train/"
IMG_DIR_TEST = "test/"

In [3]:
os.getcwd()

'C:\\Users\\bbaah'

In [4]:
os.chdir("C:/Users/bbaah/OneDrive/Desktop/Current Students/Darlene Echols/Project/")

In [5]:
os.getcwd()

'C:\\Users\\bbaah\\OneDrive\\Desktop\\Current Students\\Darlene Echols\\Project'

In [6]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
train_data.head()

,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus
0,ID00007637202177411956430,-4,2315,58.253649,79,Male,Ex-smoker
1,ID00007637202177411956430,5,2214,55.712129,79,Male,Ex-smoker
2,ID00007637202177411956430,7,2061,51.862104,79,Male,Ex-smoker
3,ID00007637202177411956430,9,2144,53.950679,79,Male,Ex-smoker
4,ID00007637202177411956430,11,2069,52.063412,79,Male,Ex-smoker


In [7]:
patients = train_data["Patient"]

In [8]:
patients.head()

0    ID00007637202177411956430
1    ID00007637202177411956430
2    ID00007637202177411956430
3    ID00007637202177411956430
4    ID00007637202177411956430
Name: Patient, dtype: object

In [9]:
PATIENT_ID = train_data["Patient"][0]    
print("Patient : ", PATIENT_ID)
print("Number of FVC observations : ", len(train_data[train_data["Patient"] == PATIENT_ID]))
print("Age : ", (train_data[train_data["Patient"] == PATIENT_ID]["Age"].values[0]))
print("Sex : ", (train_data[train_data["Patient"] == PATIENT_ID]["Sex"].values[0]))
print("SmokingStatus : ", (train_data[train_data["Patient"] == PATIENT_ID]["SmokingStatus"].values[0]))

Patient :  ID00007637202177411956430
Number of FVC observations :  9
Age :  79
Sex :  Male
SmokingStatus :  Ex-smoker


In [10]:
def natural_sort(l): 
    convert = lambda text: int(text) if text.isdigit() else text.lower() 
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(l, key = alphanum_key)

fig = plt.figure()

img_names = []
for dirictory,_,img in os.walk(IMG_DIR_TRAIN + train_data["Patient"][0]):
    img_names.append(img)

img_names = natural_sort(img_names[0])

images = []
k = 0
for i in img_names:
    images.append([plt.imshow(pydicom.dcmread(IMG_DIR_TRAIN + PATIENT_ID + "/" + i).pixel_array, cmap=plt.cm.bone)])
    k += 1
    
ani = animation.ArtistAnimation(fig, images)
plt.close()

HTML('<center>' + ani.to_html5_video() + '</center>')

In [12]:
FVC = train_data[train_data["Patient"] == PATIENT_ID]["FVC"]
Week = train_data[train_data["Patient"] == PATIENT_ID]["Weeks"]

fig = px.line(x=Week, y=FVC, title='FVC over time of patient with id ' + PATIENT_ID)

fig.update_layout(
    xaxis=dict(title = "Week"),
    yaxis=dict(title = "FVC"),
    plot_bgcolor='white'
)

fig.add_shape(
            type="line",
            x0=0,
            y0=min(FVC),
            x1=0,
            y1=max(FVC),
            line=dict(
                color="Red",
                width=2,
                dash="dashdot",
            ),
    )
py.offline.iplot(fig)

In [13]:
print("Number of patients: ", len(train_data["Patient"].unique()))
l1 = list(train_data["SmokingStatus"].unique())
smokers = ""
for i in l1:
    smokers = smokers + i + ", "
print("Among them: ", smokers[:-2])
min_Age = min(train_data["Age"].unique())
max_Age = max(train_data["Age"].unique())
print("Ages vary from ", min_Age," to ",max_Age)

Number of patients:  176
Among them:  Ex-smoker, Never smoked, Currently smokes
Ages vary from  49  to  88


In [14]:
train_data["dummy"] = 1
fig = px.bar(train_data.drop_duplicates(subset=["Patient"])[["Sex","Age","dummy"]].groupby(["Sex","Age"]).sum().reset_index().rename(columns={"dummy":"Count"}), x="Age", y="Count",color = "Sex")
fig.update_layout(
    plot_bgcolor='white'
)
py.offline.iplot(fig)

In [15]:
fig = px.bar(train_data.drop_duplicates(subset=["Patient"])[["Sex","SmokingStatus","dummy"]].groupby(["Sex","SmokingStatus"]).sum().reset_index().rename(columns={"dummy":"Count"}), x="SmokingStatus", y="Count",color = "Sex")
fig.update_layout(
    plot_bgcolor='white'
)
py.offline.iplot(fig)

In [16]:
fig = go.Figure()
Pat_Ids = train_data["Patient"].unique()
for i in Pat_Ids[:15]:
    fig.add_trace(go.Scatter(
            x=train_data[train_data["Patient"]==i]["Weeks"],
            y=train_data[train_data["Patient"]==i]["FVC"],
            name = i
        ))
fig.update_layout(
    plot_bgcolor='white'
)
fig.add_shape(
            type="line",
            x0=0,
            y0=1000,
            x1=0,
            y1=5000,
            line=dict(
                color="Red",
                width=2,
                dash="dashdot",
            ),
    )
py.offline.iplot(fig)

In [17]:
fig = px.scatter_matrix(train_data, dimensions=["Weeks", "FVC", "Percent", "Age"], color="Sex")
fig.update_layout(
    plot_bgcolor='white'
)
py.offline.iplot(fig)

In [19]:
num_files = []
for i in Pat_Ids:
    num_files.append(len([name for name in os.listdir(IMG_DIR_TRAIN + i + '/') if os.path.isfile(os.path.join(IMG_DIR_TRAIN + i + '/', name))]))
fig = go.Figure(go.Bar(name='SF Zoo',x=Pat_Ids,y=num_files))
fig.update_layout(
    plot_bgcolor='white'
)
py.offline.iplot(fig)

In [20]:
train_data = train_data.drop(columns =["dummy"])

KeyError: "['dummy'] not found in axis"

In [21]:
class DataGenCT(Sequence):
    
    def __init__(self, patients, dataset, cols, batch_size=32, train = 1):
        
        self.patients = [i for i in patients if i not in ['ID00011637202177653955184', 'ID00052637202186188008618']]
        self.dataset = dataset
        self.batch_size = batch_size
        self.cols = cols
        self.patient_scans = {}
        self.train = train
        IMG_DIR_TRAIN = "train/"
        IMG_DIR_TEST = "test/"
        if train:
            self.IMG_DIR = IMG_DIR_TRAIN
        else:
            self.IMG_DIR = IMG_DIR_TEST
        
        for patient in patients:
            self.patient_scans[patient] = natural_sort([i for i in os.listdir(self.IMG_DIR + patient + "/")])
    
    def __len__(self):
        return 1100

    def __getitem__(self,idx):
        CT_Scan = []
        Answer, Table = [], [] 
        
        keys = np.random.choice(self.patients, size = self.batch_size)
        for key in keys:
            try:
                idx = np.random.choice(self.patient_scans[key], size=1)[0]
                dataset_copy = self.dataset[self.dataset["Patient"] == key]
                rand_week = random.choice(list(dataset_copy["Weeks"]))

                img = pydicom.dcmread(self.IMG_DIR + key + "/" + idx).pixel_array
                img_min = img.min()
                img_max = img.max()
                img = cv2.resize((img - img_min) / (img_max - img_min), (IMAGE_SIZE, IMAGE_SIZE))
                CT_Scan.append(img)
                Answer.append(dataset_copy[dataset_copy["Weeks"] == rand_week]["FVC"].values[0])
                Table.append(dataset_copy[dataset_copy["Weeks"] == rand_week][self.cols].values[0])
            except Exception as e:
                continue

        CT_Scan = np.expand_dims(np.array(CT_Scan), axis=-1)
        return [CT_Scan, np.array(Table)] , np.array(Answer)

In [22]:
le_sex = LabelEncoder()
le_smoke = LabelEncoder()

In [23]:
le_sex = le_sex.fit(train_data["Sex"])
train_data["Sex"] = le_sex.transform(train_data["Sex"])
le_smoke = le_smoke.fit(train_data["SmokingStatus"])
train_data["SmokingStatus"] = le_smoke.transform(train_data["SmokingStatus"])
test_data["Sex"] = le_sex.transform(test_data["Sex"])
test_data["SmokingStatus"] = le_smoke.transform(test_data["SmokingStatus"])

In [24]:
fig = ff.create_distplot([train_data["Weeks"].values], ['Weeks distribution'], show_rug=False)
fig.update_layout(
    plot_bgcolor='white'
)
py.offline.iplot(fig)

In [28]:
fig = ff.create_distplot([train_data["Percent"].values], ['Percent distribution'], show_rug=False)
fig.update_layout(
    plot_bgcolor='white'
)
py.offline.iplot(fig)

In [29]:
fig = ff.create_distplot([train_data["Age"].values], ['Age distribution'], show_rug=False)
fig.update_layout(
    plot_bgcolor='white'
)
py.offline.iplot(fig)

In [30]:
transformer_weeks = RobustScaler().fit(np.array(train_data["Weeks"]).reshape(-1, 1))
train_data["Weeks"] = transformer_weeks.transform(np.array(train_data["Weeks"]).reshape(-1, 1)).reshape(1,-1)[0]
transformer_perc = RobustScaler().fit(np.array(train_data["Percent"]).reshape(-1, 1))
train_data["Percent"] = transformer_perc.transform(np.array(train_data["Percent"]).reshape(-1, 1)).reshape(1,-1)[0]
transformer_age = RobustScaler().fit(np.array(train_data["Age"]).reshape(-1, 1))
train_data["Age"] = transformer_age.transform(np.array(train_data["Age"]).reshape(-1, 1)).reshape(1,-1)[0]

In [31]:
test_data["Weeks"] = transformer_weeks.transform(np.array(test_data["Weeks"]).reshape(-1, 1)).reshape(1,-1)[0]
test_data["Percent"] = transformer_perc.transform(np.array(test_data["Percent"]).reshape(-1, 1)).reshape(1,-1)[0]
test_data["Age"] = transformer_age.transform(np.array(test_data["Age"]).reshape(-1, 1)).reshape(1,-1)[0]

In [32]:
train_data.head()

,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus
0,ID00007637202177411956430,-0.914286,2315,-0.675626,1.222222,1,1
1,ID00007637202177411956430,-0.657143,2214,-0.774179,1.222222,1,1
2,ID00007637202177411956430,-0.600000,2061,-0.923472,1.222222,1,1
3,ID00007637202177411956430,-0.542857,2144,-0.842483,1.222222,1,1
4,ID00007637202177411956430,-0.485714,2069,-0.915666,1.222222,1,1


In [33]:
test_data.head()

,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus
0,ID00419637202311204720264,-0.628571,3020,-0.212890,0.555556,1,1
1,ID00421637202311550012437,-0.371429,2739,0.246947,0.000000,1,1
2,ID00422637202311677017371,-0.628571,1930,0.038605,0.555556,1,1
3,ID00423637202312137826377,-0.314286,3294,0.138899,0.444444,1,1
4,ID00426637202313170790466,-0.800000,2925,-0.149368,0.555556,1,2


In [34]:
from tensorflow.keras.models import Sequential

In [35]:
cnn = Sequential()

In [36]:
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D

In [38]:
IMAGE_SIZE = 224
start1 = Input(shape=(5,),name = "Tab_input")
start2 = Input(shape=(IMAGE_SIZE, IMAGE_SIZE,1), name = "Image_input")

In [40]:
def identity_block(input_tensor, filters):
  
    filters1, filters2, filters3 = filters


    x = Conv2D(filters1, (1, 1))(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1))(x)
    x = BatchNormalization()(x)

    x = add([x, input_tensor])
    x = Activation('relu')(x)
    return x

def conv_block(input_tensor, filters):
   
    filters1, filters2, filters3 = filters


    x = Conv2D(filters1, (1, 1), strides=(2, 2))(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1))(x)
    x = BatchNormalization()(x)

    shortcut = Conv2D(filters3, (1, 1), strides=(2, 2))(input_tensor)
    shortcut = BatchNormalization()(shortcut)

    x = add([x, shortcut])
    x = Activation('relu')(x)
    return x

In [41]:
x = ZeroPadding2D((3, 3))(start2)
x = Conv2D(64, (7, 7), strides=(2, 2))(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((3, 3), strides=(2, 2))(x)

x = conv_block(x, [64, 64, 256])
x = identity_block(x, [64, 64, 256])
x = identity_block(x, [64, 64, 256])

x = conv_block(x, [128, 128, 512])
x = identity_block(x, [128, 128, 512])
x = identity_block(x, [128, 128, 512])
x = identity_block(x, [128, 128, 512])

x = AveragePooling2D((7, 7))(x)
x = Flatten()(x)
x1 = Dense(100, activation="relu")(start1)
x1 = Dense(100, activation="relu")(x1)
x = concatenate([x, x1])
x = Dense(50, activation="relu")(x)

out = Dense(3, activation='relu',)(x)
model = Model([start2, start1], out)

In [49]:
C1 = tf.constant(70, dtype='float32')
C2 = tf.constant(1000, dtype='float32')
quantiles = [.15, .50, .85]

def metric(y_true, y_pred, Sigma):
    Sigma_clipped = np.clip(Sigma, 70, 9e9)  
    Delta = np.clip(np.abs(y_true - y_pred), 0 , 1000)  
    return np.mean(-1 * (np.sqrt(2) * Delta / Sigma_clipped) - np.log(np.sqrt(2) * Sigma_clipped))

def FVC_score(y_true, y_pred):
    y_true = tf.dtypes.cast(y_true, tf.float32)
    y_pred = tf.dtypes.cast(y_pred, tf.float32)
    sigma = y_pred[:, 2] - y_pred[:, 1]
    fvc_pred = y_pred[:, 1]
    Sigma_clipped = tf.maximum(sigma, C1)
    Delta = tf.abs(y_true[:, 0] - fvc_pred)
    Delta = tf.minimum(Delta, C2)
    sq2 = tf.sqrt(tf.dtypes.cast(2, dtype=tf.float32))
    metric = sq2 * (Delta / Sigma_clipped) * sq2 + tf.math.log(Sigma_clipped * sq2)
    return K.mean(metric)

def Quantile_loss(y_true, y_pred):
    q = tf.constant(np.array([quantiles]), dtype=tf.float32)
    y_true = tf.dtypes.cast(y_true, tf.float32)
    y_pred = tf.dtypes.cast(y_pred, tf.float32)
    e = y_true - y_pred
    v = tf.maximum(q * e, (q - 1) * e)
    return K.mean(v)

def model_loss():
    def loss(y_true, y_pred):
        lambd = 0.8
        return lambd * Quantile_loss(y_true, y_pred) + (1 - lambd) * FVC_score(y_true, y_pred)
    return loss

In [50]:
Test_generator = DataGenCT(patients=Pat_Ids[150:len(Pat_Ids)] ,
                            dataset = train_data,
                            cols= ["Weeks","Percent","Age","Sex","SmokingStatus"],
                          )


Train_generator = DataGenCT(patients=Pat_Ids[0:150],
                            dataset = train_data,
                            cols= ["Weeks","Percent","Age","Sex","SmokingStatus"])

In [51]:
model.compile(loss=model_loss(), 
              optimizer=tf.keras.optimizers.Adam(lr=0.1, beta_1=0.9, beta_2=0.999, 
                                                 epsilon=None, decay=0.01, amsgrad=False), 
              metrics=[FVC_score])

In [52]:
history = model.fit_generator(Train_generator , 
                    steps_per_epoch = 100,
                    epochs = 10,
                    validation_data = Test_generator,
                    use_multiprocessing = False,
                    workers = 1,
                    validation_steps = 20,
                    verbose=1
                             )

Epoch 1/10
100/100 [==============================] - 881s 9s/step - loss: 129.7116 - FVC_score: 8.7598 - val_loss: 11307.0154 - val_FVC_score: 9.4379
Epoch 2/10
100/100 [==============================] - 875s 9s/step - loss: nan - FVC_score: nan - val_loss: nan - val_FVC_score: nan
Epoch 3/10
100/100 [==============================] - 981s 10s/step - loss: nan - FVC_score: nan - val_loss: nan - val_FVC_score: nan
Epoch 4/10
100/100 [==============================] - 875s 9s/step - loss: nan - FVC_score: nan - val_loss: nan - val_FVC_score: nan
Epoch 5/10
100/100 [==============================] - 1021s 10s/step - loss: nan - FVC_score: nan - val_loss: nan - val_FVC_score: nan
Epoch 6/10
100/100 [==============================] - 1041s 10s/step - loss: nan - FVC_score: nan - val_loss: nan - val_FVC_score: nan
Epoch 7/10
100/100 [==============================] - 1115s 11s/step - loss: nan - FVC_score: nan - val_loss: nan - val_FVC_score: nan
Epoch 8/10
100/100 [========================

In [32]:
cnn.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', 
               input_shape=(28, 28, 1)))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [33]:
cnn.add(MaxPooling2D(pool_size=(2, 2)))

In [34]:
cnn.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))

In [35]:
cnn.add(MaxPooling2D(pool_size=(2, 2)))

In [36]:
cnn.add(Flatten())

In [37]:
cnn.add(Dense(units=128, activation='relu'))

In [38]:
cnn.add(Dense(units=10, activation='softmax'))

In [39]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 3200)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               409728    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1

In [40]:
cnn.compile(optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy'])

In [41]:
from tensorflow.keras.callbacks import TensorBoard

import time

tensorboard_callback = TensorBoard(log_dir=f'./logs/mnist{time.time()}',   
    histogram_freq=1, write_graph=True)

In [48]:
X_train =train_data['Percent']

In [47]:
y_train.head()

0    2315
1    2214
2    2061
3    2144
4    2069
Name: FVC, dtype: int64

In [49]:
X_train.head()

0   -0.675626
1   -0.774179
2   -0.923472
3   -0.842483
4   -0.915666
Name: Percent, dtype: float64

In [50]:
y_train = train_data['FVC']

In [51]:
import statsmodels.api as sm

X = X_train
y = y_train

# Note the difference in argument order
model = sm.OLS(y, X).fit()
predictions = model.predict(X) # make the predictions by the model

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                    FVC   R-squared (uncentered):                   0.086
Model:                            OLS   Adj. R-squared (uncentered):              0.085
Method:                 Least Squares   F-statistic:                              145.8
Date:                Tue, 25 Aug 2020   Prob (F-statistic):                    3.87e-32
Time:                        14:51:49   Log-Likelihood:                         -14432.
No. Observations:                1549   AIC:                                  2.887e+04
Df Residuals:                    1548   BIC:                                  2.887e+04
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Percent     1069.7324     88.605     12.073      0.000     895.935    1243.530
==============================================================================
Omnibus:                      399.794   Durbin-Watson:                   0.014
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              888.797
Skew:                          -1.443   Prob(JB):                    1.00e-193
Kurtosis:                       5.332   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [52]:
img_names

['1.dcm',
 '2.dcm',
 '3.dcm',
 '4.dcm',
 '5.dcm',
 '6.dcm',
 '7.dcm',
 '8.dcm',
 '9.dcm',
 '10.dcm',
 '11.dcm',
 '12.dcm',
 '13.dcm',
 '14.dcm',
 '15.dcm',
 '16.dcm',
 '17.dcm',
 '18.dcm',
 '19.dcm',
 '20.dcm',
 '21.dcm',
 '22.dcm',
 '23.dcm',
 '24.dcm',
 '25.dcm',
 '26.dcm',
 '27.dcm',
 '28.dcm',
 '29.dcm',
 '30.dcm']

In [55]:
images

[[<matplotlib.image.AxesImage at 0x1fd75918b70>],

In [57]:
        Test_generator = DataGenCT(patients=Pat_Ids[150:len(Pat_Ids)] ,
                            dataset = train_data,
                            cols= ["Weeks","Percent","Age","Sex","SmokingStatus"],
                          )


Train_generator = DataGenCT(patients=Pat_Ids[0:150],
                            dataset = train_data,
                            cols= ["Weeks","Percent","Age","Sex","SmokingStatus"])

In [61]:
patient = Test_generator.__getitem__('ID00007637202177411956430')

In [62]:
patient

([array([], shape=(0, 1), dtype=float64), array([], dtype=float64)],
 array([], dtype=float64))

In [63]:
patient[0]

[array([], shape=(0, 1), dtype=float64), array([], dtype=float64)]

In [64]:
from tensorflow.keras.models import Sequential

In [65]:
rnn = Sequential()

In [66]:
from tensorflow.keras.layers import Dense, LSTM, Embedding

In [67]:
number_of_params=485514

In [82]:
rnn.add(Embedding(input_dim=number_of_params, output_dim=128,
                  input_length=200))

In [83]:
rnn.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))

In [84]:
rnn.compile(optimizer='adam',
            loss='binary_crossentropy', 
            metrics=['accuracy'])

In [85]:
rnn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 128)          62145792  
_________________________________________________________________
lstm (LSTM)                  (None, 128)               131584    
_________________________________________________________________
embedding_1 (Embedding)      (None, 128, 128)          62145792  
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
embedding_2 (Embedding)      (None, 128, 128)          62145792  
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
Total params: 186,832,128
Trainable params: 186,832,128
Non-trainable params: 0
________________________________________

In [86]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [79]:
X_val = test_data['Patient']

In [80]:
y_val = test_data['FVC']

In [81]:
rnn.fit(Train_generator, y_train, epochs=10, batch_size=32, 
        validation_data=(Test_generator, y_val))

ValueError: Error when checking input: expected embedding_input to have shape (200,) but got array with shape (1,)